In [1]:
import cv2
import numpy as np

# Capture video and initialize background subtractor
cap = cv2.VideoCapture('Images/traffic.mp4')
bg_subtractor = cv2.createBackgroundSubtractorKNN()

# Lucas-Kanade parameters
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read initial frame and set up points for tracking
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

kernel = np.ones((9, 9), dtype=np.uint8)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Background subtraction mask
    fg_mask = bg_subtractor.apply(gray)

    # Calculate optical flow for Lucas-Kanade
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

    # Create a mask for optical flow points
    flow_mask = np.zeros_like(fg_mask)
    if p1 is not None:
        good_new = p1[st == 1]
        for point in good_new:
            x, y = point.ravel()
            flow_mask = cv2.circle(flow_mask, (int(x), int(y)), 5, 255, -1)  # Draw points as white on black mask

    # Merge masks
    combined_mask = cv2.bitwise_or(fg_mask, flow_mask)

    # Refine the mask (optional)
    kernel = np.ones((3, 3), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)

    # Remove very small bright areas
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)


    # # Threshhold the mask so only the bright areas are detected
    _, combined_mask = cv2.threshold(combined_mask, 200, 255, cv2.THRESH_BINARY)

    # Display the result
    cv2.imshow('Combined Mask', combined_mask)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update previous frame and points
    old_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2)




cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

2024-11-06 16:01:18.269 python[64651:4534076] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-06 16:01:18.269 python[64651:4534076] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


-1